## SRJ - Trying for H2

In [19]:
from pennylane import qchem
import pennylane as qml
from jax import numpy as jnp
import jax

jax.config.update("jax_enable_x64", True)

electrons = 2

symbols  = ['H', 'H']
geometry = jnp.array([[0.0, 0.0, -0.69434785],
                     [0.0, 0.0,  0.69434785]])

mol = qchem.Molecule(symbols, geometry)
H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-3g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)
print(hf_state)

from pennylane.fermi import from_string

singles, doubles = qchem.excitations(electrons, qubits)

singles_fermi = []
for ex in singles:
    singles_fermi.append(from_string(f"{ex[1]}+ {ex[0]}-")
                       - from_string(f"{ex[0]}+ {ex[1]}-"))

doubles_fermi = []
for ex in doubles:
    print(ex)
    doubles_fermi.append(from_string(f"{ex[3]}+ {ex[2]}+ {ex[1]}- {ex[0]}-")
                       - from_string(f"{ex[0]}+ {ex[1]}+ {ex[2]}- {ex[3]}-"))
print(doubles_fermi)



[1 1 0 0]
[0, 1, 2, 3]
[FermiSentence({FermiWord({(0, 3): '+', (1, 2): '+', (2, 1): '-', (3, 0): '-'}): 1.0, FermiWord({(0, 0): '+', (1, 1): '+', (2, 2): '-', (3, 3): '-'}): -1.0})]


In [34]:
singles_pauli = []
for op in singles_fermi:
    print('Op operator:', op)
    singles_pauli.append(qml.jordan_wigner(op, ps=True))


doubles_pauli = []
for op in doubles_fermi:
    doubles_pauli.append(qml.fermi.jordan_wigner(op, ps=True))

print(doubles_pauli)

Op operator: 1.0 * a⁺(2) a(0)
+ -1.0 * a⁺(0) a(2)
Op operator: 1.0 * a⁺(3) a(1)
+ -1.0 * a⁺(1) a(3)
[0.125j * Y(2) @ X(3) @ X(0) @ X(1)
+ -0.125j * Y(2) @ X(3) @ Y(0) @ Y(1)
+ -0.125j * X(2) @ X(3) @ Y(0) @ X(1)
+ -0.125j * X(2) @ X(3) @ X(0) @ Y(1)
+ 0.125j * Y(2) @ Y(3) @ Y(0) @ X(1)
+ 0.125j * Y(2) @ Y(3) @ X(0) @ Y(1)
+ 0.125j * X(2) @ Y(3) @ X(0) @ X(1)
+ -0.125j * X(2) @ Y(3) @ Y(0) @ Y(1)]


In [41]:
params = jnp.array([0.22347661, 0.0, 0.0])
theta = 0.22347661
dev = qml.device("default.qubit", wires=qubits)

@qml.qnode(dev)
def circuit(params):
    qml.BasisState(hf_state, wires=range(qubits))

    #for i, excitation in enumerate(doubles_pauli):
    #    print('The exc.:', excitation)
    #    print('The params:', params[i])
        #qml.exp((excitation * params[i] / 2).operation()), range(qubits)
    
    for i in range(len(doubles_pauli)):
        print('The doubles:', doubles_pauli[i])
        print('The params:', theta)
        qml.exp((doubles_pauli[i] * theta/2).operation()), range(qubits)

    #for j, excitation in enumerate(singles_pauli):
    #    qml.exp((excitation * params[i + j + 1] / 2).operation()), range(qubits)

    return qml.expval(H)

print('Energy =', circuit(params))

The doubles: 0.125j * Y(2) @ X(3) @ X(0) @ X(1)
+ -0.125j * Y(2) @ X(3) @ Y(0) @ Y(1)
+ -0.125j * X(2) @ X(3) @ Y(0) @ X(1)
+ -0.125j * X(2) @ X(3) @ X(0) @ Y(1)
+ 0.125j * Y(2) @ Y(3) @ Y(0) @ X(1)
+ 0.125j * Y(2) @ Y(3) @ X(0) @ Y(1)
+ 0.125j * X(2) @ Y(3) @ X(0) @ X(1)
+ -0.125j * X(2) @ Y(3) @ Y(0) @ Y(1)
The params: 0.22347661
Energy = -1.1373060512054298
